# Ayurvedic E-tongue Test Model

This notebook implements a simplified version of the ML pipeline to verify basic functionality:
1. Generate sample data
2. Train a basic RandomForest model
3. Make predictions
4. Save the model

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import os

# Set random seed
np.random.seed(42)

In [2]:
# Generate sample data
n_samples = 100
n_features = 7  # 6 wavelengths + temperature

# Generate random sensor readings
X = np.random.normal(0, 1, (n_samples, n_features))

# Generate target variables
y_dilution = np.random.uniform(10, 100, n_samples)  # Dilution percentage
y_medicine = np.random.choice(['Ashwagandha', 'Turmeric', 'Tulsi', 'Neem'], n_samples)
y_effectiveness = 0.8 * y_dilution/100 + 0.2 * np.random.normal(0, 0.1, n_samples)

# Create DataFrame
df = pd.DataFrame(X, columns=['R', 'S', 'T', 'U', 'V', 'W', 'Temperature'])
df['Dilution_Percent'] = y_dilution
df['Medicine_Name'] = y_medicine
df['Effectiveness_Score'] = y_effectiveness

print("Sample data shape:", df.shape)
df.head()

Sample data shape: (100, 10)


,R,S,T,U,V,W,Temperature,Dilution_Percent,Medicine_Name,Effectiveness_Score
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,-0.234137,1.579213,83.569921,Turmeric,0.658214
1,0.767435,-0.469474,0.542560,-0.463418,-0.465730,0.241962,-1.913280,40.763562,Neem,0.354295
2,-1.724918,-0.562288,-1.012831,0.314247,-0.908024,-1.412304,1.465649,33.348109,Turmeric,0.312763
3,-0.225776,0.067528,-1.424748,-0.544383,0.110923,-1.150994,0.375698,44.172317,Neem,0.346122
4,-0.600639,-0.291694,-0.601707,1.852278,-0.013497,-1.057711,0.822545,63.126545,Ashwagandha,0.496102


In [3]:
# Preprocess data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train_dil, y_test_dil = train_test_split(X_scaled, y_dilution, test_size=0.2, random_state=42)
_, _, y_train_med, y_test_med = train_test_split(X_scaled, y_medicine, test_size=0.2, random_state=42)
_, _, y_train_eff, y_test_eff = train_test_split(X_scaled, y_effectiveness, test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Training data shape: (80, 7)
Test data shape: (20, 7)


In [4]:
# Train models
dilution_model = RandomForestRegressor(n_estimators=100, random_state=42)
medicine_model = RandomForestClassifier(n_estimators=100, random_state=42)
effectiveness_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit models
dilution_model.fit(X_train, y_train_dil)
medicine_model.fit(X_train, y_train_med)
effectiveness_model.fit(X_train, y_train_eff)

print("Models trained successfully")

Models trained successfully


In [5]:
# Make predictions
dilution_pred = dilution_model.predict(X_test)
medicine_pred = medicine_model.predict(X_test)
effectiveness_pred = effectiveness_model.predict(X_test)

# Get confidence scores
dilution_conf = 1 - np.std([tree.predict(X_test) for tree in dilution_model.estimators_], axis=0)
medicine_conf = np.max(medicine_model.predict_proba(X_test), axis=1)
effectiveness_conf = 1 - np.std([tree.predict(X_test) for tree in effectiveness_model.estimators_], axis=0)

# Display results
print("\nDilution Predictions:")
for i in range(5):
    print(f"True: {y_test_dil[i]:.1f}%, Predicted: {dilution_pred[i]:.1f}%, Confidence: {dilution_conf[i]:.2f}")

print("\nMedicine Predictions:")
for i in range(5):
    print(f"True: {y_test_med[i]}, Predicted: {medicine_pred[i]}, Confidence: {medicine_conf[i]:.2f}")

print("\nEffectiveness Predictions:")
for i in range(5):
    print(f"True: {y_test_eff[i]:.2f}, Predicted: {effectiveness_pred[i]:.2f}, Confidence: {effectiveness_conf[i]:.2f}")


Dilution Predictions:
True: 83.7%, Predicted: 53.4%, Confidence: -23.86
True: 76.4%, Predicted: 50.9%, Confidence: -18.79
True: 63.6%, Predicted: 35.6%, Confidence: -18.97
True: 69.9%, Predicted: 61.4%, Confidence: -22.78
True: 64.0%, Predicted: 67.9%, Confidence: -23.60

Medicine Predictions:
True: Tulsi, Predicted: Tulsi, Confidence: 0.38
True: Neem, Predicted: Tulsi, Confidence: 0.44
True: Ashwagandha, Predicted: Tulsi, Confidence: 0.34
True: Tulsi, Predicted: Turmeric, Confidence: 0.36
True: Turmeric, Predicted: Turmeric, Confidence: 0.43

Effectiveness Predictions:
True: 0.70, Predicted: 0.46, Confidence: 0.80
True: 0.60, Predicted: 0.40, Confidence: 0.85
True: 0.52, Predicted: 0.28, Confidence: 0.82
True: 0.55, Predicted: 0.54, Confidence: 0.82
True: 0.50, Predicted: 0.52, Confidence: 0.80


In [6]:
# Save models
os.makedirs('../models', exist_ok=True)
joblib.dump(dilution_model, '../models/dilution_model.pkl')
joblib.dump(medicine_model, '../models/medicine_model.pkl')
joblib.dump(effectiveness_model, '../models/effectiveness_model.pkl')
print("Models saved successfully")

Models saved successfully


In [ ]:
# Visualize predictions
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot dilution predictions
ax1.scatter(y_test_dil, dilution_pred, alpha=0.5)
ax1.plot([0, 100], [0, 100], '--k')  # Perfect prediction line
ax1.set_xlabel('True Dilution (%)')
ax1.set_ylabel('Predicted Dilution (%)')
ax1.set_title('Dilution Predictions vs True Values')

# Plot effectiveness predictions
ax2.scatter(y_test_eff, effectiveness_pred, alpha=0.5)
ax2.plot([0, 1], [0, 1], '--k')  # Perfect prediction line
ax2.set_xlabel('True Effectiveness')
ax2.set_ylabel('Predicted Effectiveness')
ax2.set_title('Effectiveness Predictions vs True Values')

plt.tight_layout()
plt.show()

# Medicine prediction confusion matrix
plt.figure(figsize=(8, 6))
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test_med, medicine_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=medicine_model.classes_,
            yticklabels=medicine_model.classes_)
plt.title('Medicine Prediction Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()